In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px


In [2]:
df = pd.read_csv("log_fw_3.csv", sep=";", header=0)
df = df.sample(n = 100000)

In [3]:
df['portsrc'] = df['portsrc'].values.astype(str)
df['portdst'] = df['portdst'].values.astype(str)
df['policyid'] = df['policyid'].values.astype(str)
df['numproto'] = df['numproto'].values.astype(str)
df['datetime'] = pd.to_datetime(df['datetime'])


In [4]:
print(df.info())
print(df.head(3))
df[df["action"]=="DENY"]["policyid"].value_counts()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 558809 to 678385
Data columns (total 9 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   datetime  100000 non-null  datetime64[ns]
 1   ipsrc     100000 non-null  object        
 2   ipdst     100000 non-null  object        
 3   proto     100000 non-null  object        
 4   portsrc   100000 non-null  object        
 5   portdst   100000 non-null  object        
 6   policyid  100000 non-null  object        
 7   action    100000 non-null  object        
 8   numproto  100000 non-null  object        
dtypes: datetime64[ns](1), object(8)
memory usage: 7.6+ MB
None
                  datetime            ipsrc        ipdst proto  portsrc  \
558809 2023-02-21 22:16:00     24.21.253.96  17.17.17.17   TCP   1933.0   
648696 2023-02-23 01:04:00  109.234.162.235  17.17.17.17   TCP  56642.0   
878465 2023-02-28 12:30:00   198.46.189.193  17.17.17.17   TCP  60684.0   

   

999.0    21498
7.0       1188
6.0        534
5.0        118
Name: policyid, dtype: int64

In [5]:
df_tcp = df.loc[df['proto'] == 'TCP']


In [6]:
df_grouped = df_tcp.groupby(['policyid', 'portdst', 'action']).size().reset_index(name='count')


In [7]:
print(df_grouped.sort_values(by=['count'], ascending=False).head(5))
print(df_grouped.info())


   policyid  portdst  action  count
0       1.0    443.0  PERMIT  19620
28    555.0  60758.0  PERMIT   8570
22    555.0  58428.0  PERMIT   8487
10    555.0  52080.0  PERMIT   6537
14    555.0  52691.0  PERMIT   4003
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14890 entries, 0 to 14889
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   policyid  14890 non-null  object
 1   portdst   14890 non-null  object
 2   action    14890 non-null  object
 3   count     14890 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 465.4+ KB
None


In [8]:

df_grouped_5 = df_grouped.sort_values(by=['count'], ascending=False).head(5)

# Créer un graphique en barres des actions en fonction des ports de destination
fig = go.Figure(data=[go.Bar(x=df_grouped_5.loc[df_grouped_5['action'] == 'PERMIT']['portdst'], 
                             y=df_grouped_5.loc[df_grouped_5['action'] == 'PERMIT']['count'], 
                             name='PERMIT'),
                      go.Bar(x=df_grouped_5.loc[df_grouped_5['action'] == 'DENY']['portdst'], 
                             y=df_grouped_5.loc[df_grouped_5['action'] == 'DENY']['count'], 
                             name='DENY')])

# Ajouter un titre et des labels d'axe
fig.update_layout(title='Actions en fonction des ports de destination', 
                  xaxis_title='Ports de destination', 
                  yaxis_title='Nombre de règles')

# Afficher le graphique
fig.show()

In [9]:
action_counts = df_tcp['action'].value_counts()
fig1 = go.Figure(go.Bar(x=action_counts.index, y=action_counts.values))
fig1.show()

In [10]:

df_filtered = df_tcp[['policyid', 'portdst', 'action']]



In [11]:
df_grouped = df_filtered.groupby(['policyid', 'portdst', 'action']).size().reset_index(name='count')


In [12]:
# Sunburst pour les données DENY
df_deny = df_grouped[df_grouped['action'] == 'DENY']


# Sunburst pour les données PERMIT
df_permit = df_grouped[df_grouped['action'] == 'PERMIT']



In [14]:
df_deny = df_tcp[df_tcp['action'] == 'DENY']
df_permit = df_tcp[df_tcp['action'] == 'PERMIT']


In [21]:
df_deny['policyid'].value_counts()

999.0    21498
7.0       1188
6.0        534
5.0        118
Name: policyid, dtype: int64

In [46]:
top_ports_deny = df_deny['portdst'].value_counts().nlargest(10).index.tolist()
df_deny = df_deny[df_deny['portdst'].isin(top_ports_deny)]
top_ports_permit = df_permit['portdst'].value_counts().nlargest(10).index.tolist()
df_permit = df_permit[df_permit['portdst'].isin(top_ports_permit)]


,datetime,ipsrc,ipdst,proto,portsrc,portdst,policyid,action,numproto
607598,2023-02-22 23:44:00,118.45.162.14,17.17.17.17,TCP,35844.0,23.0,7.0,DENY,6.0
591411,2023-02-22 04:57:00,209.141.45.200,17.17.17.17,TCP,4278.0,23.0,7.0,DENY,6.0
591160,2023-02-22 04:55:00,209.141.45.200,17.17.17.17,TCP,4278.0,23.0,7.0,DENY,6.0
599832,2023-02-22 15:12:00,182.247.155.183,17.17.17.17,TCP,54419.0,23.0,7.0,DENY,6.0
268129,2023-02-17 04:26:00,88.254.237.191,17.17.17.17,TCP,60908.0,23.0,7.0,DENY,6.0
...,...,...,...,...,...,...,...,...,...
45189,2023-02-13 05:42:00,219.138.90.85,17.17.17.17,TCP,19083.0,445.0,6.0,DENY,6.0
439903,2023-02-19 23:05:00,64.112.72.210,17.17.17.17,TCP,38722.0,23.0,7.0,DENY,6.0
267234,2023-02-17 03:27:00,103.121.18.12,17.17.17.17,TCP,58808.0,445.0,6.0,DENY,6.0
940092,2023-02-28 13:37:00,223.244.83.227,17.17.17.17,TCP,38880.0,23.0,7.0,DENY,6.0


In [48]:
df_deny_gb = df_deny.groupby(['policyid', 'portdst']).size().reset_index(name='counts')
df_permit_gb = df_permit.groupby(['policyid', 'portdst']).size().reset_index(name='counts')


In [49]:
deny_sunburst = px.sunburst(df_deny_gb, path=['policyid', 'portdst'], values='counts')
deny_sunburst.show()

In [50]:
permit_sunburst = px.sunburst(df_permit_gb, path=['policyid', 'portdst'], values='counts')
permit_sunburst.show()

In [26]:
test = pd.DataFrame(df_tcp.value_counts(['action','policyid']))
test.reset_index(inplace=True)

In [28]:
test.columns = ['action','policyid','count']

In [31]:
test_sunburst = px.sunburst(test, path=['action', 'policyid'], values='count')
test_sunburst.show()